# Run a test suite programmatically

In [1]:
EXAMPLES_FOLDER = "../examples"

In [2]:
import os, pprint
[f for f in os.listdir(EXAMPLES_FOLDER) if not f.startswith('.')]

['change_case',
 'multivariate',
 'sacurine',
 'workflow-test-suite-full.yml',
 'workflow-test-suite-min.yml',
 'workflows.json']

In [3]:
suite_conf_filename = os.path.join(EXAMPLES_FOLDER, "workflow-test-suite-min.yml")

In [4]:
import ruamel.yaml as yaml
with open(suite_conf_filename, "r") as fp:
    data = yaml.load(fp)
    print yaml.round_trip_dump(data, indent=2, block_seq_indent=3)

workflows:
  change_case:
    expected:
      OutputText: change_case/expected_output
    inputs:
      InputText: change_case/input
    file: change_case/workflow.ga
  multivariate:
    expected:
      variableMetadata_out: multivariate/variableMetadata_out
      sampleMetadata_out: multivariate/sampleMetadata_out
    inputs:
      DataMatrix: multivariate/dataMatrix.tsv
      SampleMetadata: multivariate/sampleMetadata.tsv
      VariableMetadata: multivariate/variableMetadata.tsv
    params:
      3:
        predI: '1'
        respC: gender
        orthoI: NA
        testL: 'FALSE'
    file: multivariate/workflow.ga



In [5]:
from wft4galaxy import WorkflowTestConfiguration

In [6]:
help(WorkflowTestConfiguration.load)

Help on function load in module wft4galaxy:

load(filename='workflow-test-suite.yml', workflow_test_name=None, file_format='YAML', output_folder=None)
    Load the configuration of a workflow test suite or a single workflow test
    from a YAML or JSON configuration file.
    
    :type filename: str
    :param filename: the path of the file containing the suite definition
    
    :type workflow_test_name: str
    :param workflow_test_name: the optional name of a workflow test
    
    :type file_format: str
    :param file_format: ``YAML`` or ``JSON``
    
    :rtype: dict or :class:`WorkflowTestConfiguration`
    :return: a dictionary containing the configuration of a workflow test suite or
        a single workflow test configuration (i.e., an instance of :class:`WorkflowTestConfiguration`)



In [7]:
suite_conf = WorkflowTestConfiguration.load(suite_conf_filename)

In [8]:
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(suite_conf)

{   'enable_logger': False,
    'galaxy_api_key': None,
    'galaxy_url': None,
    'output_folder': 'results',
    'workflows': {   'change_case': WorkflowTestConfig: name=change_case, file=change_case/workflow.ga, inputs=[InputText], expected_outputs=[OutputText],
                     'multivariate': WorkflowTestConfig: name=multivariate, file=multivariate/workflow.ga, inputs=[DataMatrix,SampleMetadata,VariableMetadata], expected_outputs=[variableMetadata_out,sampleMetadata_out]}}


In [9]:
pprint.pprint(suite_conf["workflows"])

{'change_case': WorkflowTestConfig: name=change_case, file=change_case/workflow.ga, inputs=[InputText], expected_outputs=[OutputText],
 'multivariate': WorkflowTestConfig: name=multivariate, file=multivariate/workflow.ga, inputs=[DataMatrix,SampleMetadata,VariableMetadata], expected_outputs=[variableMetadata_out,sampleMetadata_out]}


In [10]:
print(suite_conf["workflows"]["change_case"])

WorkflowTestConfig: name=change_case, file=change_case/workflow.ga, inputs=[InputText], expected_outputs=[OutputText]


In [11]:
from wft4galaxy import WorkflowTestSuite
suite_runner = WorkflowTestSuite()

In [12]:
suite_runner.run_test_suite(suite_conf, enable_logger=True)

Workflow Test: 'change_case' ... 2016-11-11 15:58:43,623 INFO: Create a history '_WorkflowTestHistory_569f11a6-a81f-11e6-8d9a-a45e60c4fc6b' (id: u'1e3a7ada6d556982')
2016-11-11 15:58:45,054 INFO: Workflow '_WorkflowTest_Change Case (imported from API)' (id: ddaca2bad6847b13) running ...
2016-11-11 15:58:54,386 INFO: Workflow '_WorkflowTest_Change Case (imported from API)' (id: ddaca2bad6847b13) executed
2016-11-11 15:58:54,389 INFO: Checking test output: ...
2016-11-11 15:58:54,459 INFO: Checking test output: DONE
ok
Workflow Test: 'multivariate' ... 2016-11-11 15:58:55,604 INFO: Create a history '_WorkflowTestHistory_5dc325b3-a81f-11e6-96c3-a45e60c4fc6b' (id: u'1343954bc98f8001')
2016-11-11 15:59:05,915 INFO: Workflow '_WorkflowTest_Multivariate (imported from API)' (id: 110f865fb6a7a80b) running ...
2016-11-11 15:59:21,084 INFO: Workflow '_WorkflowTest_Multivariate (imported from API)' (id: 110f865fb6a7a80b) executed
2016-11-11 15:59:21,088 INFO: Checking test output: ...
2016-11-11 

In [13]:
test_results = suite_runner.run_tests(suite_conf, enable_logger=True)

2016-11-11 15:59:29,745 INFO: Create a history '_WorkflowTestHistory_7224727d-a81f-11e6-b55e-a45e60c4fc6b' (id: u'c8a1d1385ed48f8d')
2016-11-11 15:59:31,213 INFO: Workflow '_WorkflowTest_Change Case (imported from API)' (id: 0b900c60f93c0654) running ...
2016-11-11 15:59:43,757 INFO: Workflow '_WorkflowTest_Change Case (imported from API)' (id: 0b900c60f93c0654) executed
2016-11-11 15:59:43,758 INFO: Checking test output: ...
2016-11-11 15:59:43,802 INFO: Checking test output: DONE
2016-11-11 15:59:44,882 INFO: Create a history '_WorkflowTestHistory_7b272663-a81f-11e6-896c-a45e60c4fc6b' (id: u'2d190d7d262b60f4')
2016-11-11 15:59:53,943 INFO: Workflow '_WorkflowTest_Multivariate (imported from API)' (id: 59ace41fc068d3ad) running ...
2016-11-11 16:00:08,216 INFO: Workflow '_WorkflowTest_Multivariate (imported from API)' (id: 59ace41fc068d3ad) executed
2016-11-11 16:00:08,219 INFO: Checking test output: ...
2016-11-11 16:00:08,336 INFO: Checking test output: DONE


In [14]:
print test_results

[Test 7224727d-a81f-11e6-b55e-a45e60c4fc6b: workflow _WorkflowTest_Change Case (imported from API), intputs=[InputText], outputs=[OutputText: OK], Test 7b272663-a81f-11e6-896c-a45e60c4fc6b: workflow _WorkflowTest_Multivariate (imported from API), intputs=[DataMatrix,SampleMetadata,VariableMetadata], outputs=[variableMetadata_out: OK, sampleMetadata_out: OK]]


In [15]:
for r in test_results:
    print("Test %s:\n\t - workflow: [%s] \n\t - results: %r" % (r.test_id, r.workflow.name, r.results))

Test 7224727d-a81f-11e6-b55e-a45e60c4fc6b:
	 - workflow: [_WorkflowTest_Change Case (imported from API)] 
	 - results: {u'OutputText': True}
Test 7b272663-a81f-11e6-896c-a45e60c4fc6b:
	 - workflow: [_WorkflowTest_Multivariate (imported from API)] 
	 - results: {u'variableMetadata_out': True, u'sampleMetadata_out': True}


In [16]:
a_result = test_results[0]

In [17]:
help(a_result)

Help on instance of WorkflowTestResult in module wft4galaxy:

class WorkflowTestResult
 |  Class for representing the result of a workflow test.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, test_id, workflow, inputs, outputs, output_history, expected_outputs, missing_tools, results, output_file_map, output_folder='results')
 |  
 |  __repr__(self)
 |  
 |  __str__(self)
 |  
 |  check_output(self, output)
 |      Assert whether the actual `output` is equal to the expected accordingly
 |      to its associated `comparator` function.
 |      
 |      :type output: str or dict
 |      :param output: output name
 |      
 |      :rtype: bool
 |      :return: ``True`` if the test is passed; ``False`` otherwise.
 |  
 |  check_outputs(self)
 |      Return a map of pairs <OUTPUT_NAME>:<RESULT>, where <RESULT> is ``True``
 |      if the actual `OUTPUT_NAME` is equal to the expected accordingly
 |      to its associated `comparator` function.
 |      
 |      :rtype: dict
 |      :ret

In [18]:
print "Outputs: ", a_result.results.keys()

Outputs:  [u'OutputText']


In [19]:
a_result.passed(), a_result.failed()

(True, False)

In [20]:
a_result.check_output("OutputText")

True